# 🔧 Feature Engineering: Hypothesis-Driven Features

**Objective**: Create business-meaningful features based on validated hypotheses from EDA.

---

## Feature Engineering Strategy

We create features in **three hypothesis-driven buckets**:

| Bucket | Rationale | Features |
|--------|-----------|----------|
| **Tenure & Contract** | New customers on flexible contracts churn more | Tenure buckets, contract indicators |
| **Pricing Pressure** | High charges relative to tenure increase churn risk | Normalized charges, high-charge flags |
| **Service Dependency** | More services = more "sticky" customers | Add-on count, core vs optional usage |

---

## 1. Setup & Data Loading

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("✅ Libraries loaded!")

✅ Libraries loaded!


In [3]:
# Load data
df = pd.read_csv('../data/raw/churn.csv')

# Fix TotalCharges (convert to numeric, handle blanks)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)  # New customers with tenure=0

print(f"📦 Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
df.head()

📦 Dataset: 7,043 rows × 23 columns


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0,0,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,0,0,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,3,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,0,0,Yes


---

## 2. Bucket 1: Tenure & Contract Structure

### Hypothesis Basis:
- **H1**: Month-to-month contracts have higher churn (✅ Validated)
- **H2**: Lower tenure = higher churn risk (✅ Validated)
- **H3**: Two-year contracts have lowest churn (✅ Validated)

### 2.1 Tenure Buckets

Create meaningful tenure segments based on customer lifecycle stage.

In [4]:
# Define tenure buckets based on business logic
# 0-12 months: New customers (high risk)
# 13-24 months: Developing (moderate risk)
# 25-48 months: Established (lower risk)
# 49+ months: Loyal (lowest risk)

def create_tenure_bucket(tenure):
    if tenure <= 12:
        return '1_New (0-12m)'
    elif tenure <= 24:
        return '2_Developing (13-24m)'
    elif tenure <= 48:
        return '3_Established (25-48m)'
    else:
        return '4_Loyal (49+m)'

df['tenure_bucket'] = df['tenure'].apply(create_tenure_bucket)

# View distribution
print("📊 Tenure Bucket Distribution:")
print(df['tenure_bucket'].value_counts().sort_index())

📊 Tenure Bucket Distribution:
tenure_bucket
1_New (0-12m)             2186
2_Developing (13-24m)     1024
3_Established (25-48m)    1594
4_Loyal (49+m)            2239
Name: count, dtype: int64


In [5]:
# Validate: Churn rate by tenure bucket
tenure_churn = df.groupby('tenure_bucket')['Churn'].apply(
    lambda x: (x == 'Yes').mean() * 100
).round(2)

print("\n📈 Churn Rate by Tenure Bucket:")
print(tenure_churn.sort_index())
print("\n✅ Pattern confirmed: New customers churn most, loyal customers churn least.")


📈 Churn Rate by Tenure Bucket:
tenure_bucket
1_New (0-12m)             47.44
2_Developing (13-24m)     28.71
3_Established (25-48m)    20.39
4_Loyal (49+m)             9.51
Name: Churn, dtype: float64

✅ Pattern confirmed: New customers churn most, loyal customers churn least.


### 2.2 Contract Length Indicators

Create binary flags for contract type risk levels.

In [6]:
# Contract type indicators
df['is_month_to_month'] = (df['Contract'] == 'Month-to-month').astype(int)
df['is_one_year'] = (df['Contract'] == 'One year').astype(int)
df['is_two_year'] = (df['Contract'] == 'Two year').astype(int)

# Combined: Has commitment (1yr or 2yr contract)
df['has_contract_commitment'] = ((df['Contract'] == 'One year') | 
                                  (df['Contract'] == 'Two year')).astype(int)

print("📊 Contract Indicator Distribution:")
print(f"Month-to-month: {df['is_month_to_month'].sum():,} ({df['is_month_to_month'].mean()*100:.1f}%)")
print(f"Has commitment: {df['has_contract_commitment'].sum():,} ({df['has_contract_commitment'].mean()*100:.1f}%)")

📊 Contract Indicator Distribution:
Month-to-month: 3,875 (55.0%)
Has commitment: 3,168 (45.0%)


In [7]:
# Validate: Churn rate by contract commitment
commitment_churn = df.groupby('has_contract_commitment')['Churn'].apply(
    lambda x: (x == 'Yes').mean() * 100
).round(2)

print("\n📈 Churn Rate by Contract Commitment:")
print(f"No commitment (MTM): {commitment_churn[0]:.2f}%")
print(f"Has commitment:      {commitment_churn[1]:.2f}%")


📈 Churn Rate by Contract Commitment:
No commitment (MTM): 42.71%
Has commitment:      6.76%


### 2.3 New Customer Risk Flag

Combine tenure and contract for high-risk identification.

In [8]:
# High-risk: New customer (< 12 months) on month-to-month
df['is_new_mtm_customer'] = ((df['tenure'] <= 12) & 
                              (df['Contract'] == 'Month-to-month')).astype(int)

print("📊 New MTM Customer Flag:")
print(f"Count: {df['is_new_mtm_customer'].sum():,} ({df['is_new_mtm_customer'].mean()*100:.1f}%)")

# Validate churn rate
new_mtm_churn = df.groupby('is_new_mtm_customer')['Churn'].apply(
    lambda x: (x == 'Yes').mean() * 100
).round(2)
print(f"\nChurn rate for new MTM customers: {new_mtm_churn[1]:.2f}%")
print(f"Churn rate for others: {new_mtm_churn[0]:.2f}%")

📊 New MTM Customer Flag:
Count: 1,994 (28.3%)

Churn rate for new MTM customers: 51.35%
Churn rate for others: 16.74%


---

## 3. Bucket 2: Pricing Pressure

### Hypothesis Basis:
- **H4**: Higher monthly charges = higher churn (✅ Validated)
- **H5**: Electronic check payers churn more (✅ Validated)

### 3.1 Normalized Charges (Monthly vs Tenure)

Average charge per month of tenure - measures price sensitivity.

In [9]:
# Average monthly charge (TotalCharges / tenure)
# Handle tenure=0 (new customers) - use MonthlyCharges directly
df['avg_monthly_charge'] = np.where(
    df['tenure'] > 0,
    df['TotalCharges'] / df['tenure'],
    df['MonthlyCharges']
)

print("📊 Avg Monthly Charge Stats:")
print(df['avg_monthly_charge'].describe().round(2))

📊 Avg Monthly Charge Stats:
count    7043.00
mean       64.76
std        30.19
min        13.78
25%        35.94
50%        70.34
75%        90.17
max       121.40
Name: avg_monthly_charge, dtype: float64


In [10]:
# Charge-to-tenure ratio: How much are they paying relative to how long they've been here?
# Higher ratio = potentially more price sensitive
df['charge_tenure_ratio'] = df['MonthlyCharges'] / (df['tenure'] + 1)  # +1 to avoid division by zero

print("📊 Charge-Tenure Ratio Stats:")
print(df['charge_tenure_ratio'].describe().round(2))

📊 Charge-Tenure Ratio Stats:
count    7043.00
mean        5.77
std         8.72
min         0.26
25%         1.25
50%         2.08
75%         5.95
max        80.85
Name: charge_tenure_ratio, dtype: float64


### ⚠️ Justification: `charge_tenure_ratio`

**Why this ratio exists:**
- Captures **price sensitivity at relationship stage** — a customer paying $80/month in month 2 is in a very different position than one paying $80/month in month 50
- The ratio answers: *"How much is this customer paying relative to how committed they are?"*

**How it differs from `avg_monthly_charge`:**

| Feature | Formula | What it measures |
|---------|---------|------------------|
| `avg_monthly_charge` | TotalCharges / tenure | Historical average spend per month |
| `charge_tenure_ratio` | MonthlyCharges / (tenure + 1) | Current price burden relative to relationship depth |

- `avg_monthly_charge` is backward-looking (what they paid over time)
- `charge_tenure_ratio` is forward-looking (current price pressure)

**Why it doesn't leak tenure twice:**
- We are NOT using raw tenure as a standalone feature AND this ratio — we use `tenure_bucket` (categorical) instead
- The ratio captures the **interaction** between price and commitment, which is the actual driver (high price + low commitment = flight risk)
- This is semantically different from tenure: tenure says "how long" but not "at what cost"
- In modeling, we will check feature importance and correlation to ensure no redundancy

> **Note**: If model diagnostics show high multicollinearity with tenure-based features, we may drop this in favor of simpler features. This is a hypothesis to validate, not a certainty.

### 3.2 High-Charge Flags (Quantile-Based)

In [11]:
# Calculate quantiles for MonthlyCharges
q25 = df['MonthlyCharges'].quantile(0.25)
q50 = df['MonthlyCharges'].quantile(0.50)
q75 = df['MonthlyCharges'].quantile(0.75)

print(f"📊 Monthly Charges Quantiles:")
print(f"Q25: ${q25:.2f}")
print(f"Q50: ${q50:.2f}")
print(f"Q75: ${q75:.2f}")

📊 Monthly Charges Quantiles:
Q25: $35.50
Q50: $70.35
Q75: $89.85


In [12]:
# High charge flag: Above 75th percentile
df['is_high_charge'] = (df['MonthlyCharges'] > q75).astype(int)

# Low charge flag: Below 25th percentile
df['is_low_charge'] = (df['MonthlyCharges'] < q25).astype(int)

# Charge quartile
df['charge_quartile'] = pd.qcut(df['MonthlyCharges'], q=4, labels=['Q1_Low', 'Q2', 'Q3', 'Q4_High'])

print("📊 Charge Flag Distribution:")
print(f"High charge (>Q75): {df['is_high_charge'].sum():,} ({df['is_high_charge'].mean()*100:.1f}%)")
print(f"Low charge (<Q25):  {df['is_low_charge'].sum():,} ({df['is_low_charge'].mean()*100:.1f}%)")

📊 Charge Flag Distribution:
High charge (>Q75): 1,758 (25.0%)
Low charge (<Q25):  1,759 (25.0%)


In [13]:
# Validate: Churn rate by charge quartile
charge_churn = df.groupby('charge_quartile')['Churn'].apply(
    lambda x: (x == 'Yes').mean() * 100
).round(2)

print("\n📈 Churn Rate by Charge Quartile:")
print(charge_churn)
print("\n✅ Pattern confirmed: Higher charges correlate with higher churn.")


📈 Churn Rate by Charge Quartile:
charge_quartile
Q1_Low     11.24
Q2         24.58
Q3         37.51
Q4_High    32.88
Name: Churn, dtype: float64

✅ Pattern confirmed: Higher charges correlate with higher churn.


### 3.3 Payment Risk Indicator

In [14]:
# Electronic check = higher churn risk (validated in EDA)
df['is_electronic_check'] = (df['PaymentMethod'] == 'Electronic check').astype(int)

# Auto-pay indicator (bank transfer or credit card)
df['has_auto_pay'] = df['PaymentMethod'].str.contains('automatic', case=False).astype(int)

print("📊 Payment Method Indicators:")
print(f"Electronic check: {df['is_electronic_check'].sum():,} ({df['is_electronic_check'].mean()*100:.1f}%)")
print(f"Auto-pay:         {df['has_auto_pay'].sum():,} ({df['has_auto_pay'].mean()*100:.1f}%)")

📊 Payment Method Indicators:
Electronic check: 2,365 (33.6%)
Auto-pay:         3,066 (43.5%)


---

## 4. Bucket 3: Service Dependency

### Hypothesis Basis:
- **H7**: Fiber optic users churn more than DSL (✅ Validated)
- **H8**: More add-on services = lower churn (✅ Validated)

### 4.1 Add-On Service Count

In [15]:
# Define add-on services (internet-related)
addon_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
              'TechSupport', 'StreamingTV', 'StreamingMovies']

# Count how many add-ons each customer has
df['addon_count'] = df[addon_cols].apply(
    lambda row: sum(row == 'Yes'), axis=1
)

print("📊 Add-on Count Distribution:")
print(df['addon_count'].value_counts().sort_index())

📊 Add-on Count Distribution:
addon_count
0    2219
1     966
2    1033
3    1118
4     852
5     571
6     284
Name: count, dtype: int64


In [16]:
# Validate: Churn rate by addon count
addon_churn = df.groupby('addon_count')['Churn'].apply(
    lambda x: (x == 'Yes').mean() * 100
).round(2)

print("\n📈 Churn Rate by Add-on Count:")
print(addon_churn)
print("\n✅ Pattern confirmed: More add-ons = lower churn.")


📈 Churn Rate by Add-on Count:
addon_count
0    21.41
1    45.76
2    35.82
3    27.37
4    22.30
5    12.43
6     5.28
Name: Churn, dtype: float64

✅ Pattern confirmed: More add-ons = lower churn.


### 4.2 Service Dependency Flags

In [17]:
# Has any add-on service
df['has_any_addon'] = (df['addon_count'] > 0).astype(int)

# Has multiple add-ons (3+)
df['has_multiple_addons'] = (df['addon_count'] >= 3).astype(int)

# No add-ons at all (high risk)
df['no_addons'] = (df['addon_count'] == 0).astype(int)

print("📊 Add-on Flags:")
print(f"Has any add-on:       {df['has_any_addon'].sum():,} ({df['has_any_addon'].mean()*100:.1f}%)")
print(f"Has multiple (3+):    {df['has_multiple_addons'].sum():,} ({df['has_multiple_addons'].mean()*100:.1f}%)")
print(f"No add-ons (risky):   {df['no_addons'].sum():,} ({df['no_addons'].mean()*100:.1f}%)")

📊 Add-on Flags:
Has any add-on:       4,824 (68.5%)
Has multiple (3+):    2,825 (40.1%)
No add-ons (risky):   2,219 (31.5%)


### 4.3 Core vs Optional Service Usage

In [18]:
# Core services: Phone and Internet
df['has_phone'] = (df['PhoneService'] == 'Yes').astype(int)
df['has_internet'] = (df['InternetService'] != 'No').astype(int)
df['has_both_core'] = ((df['has_phone'] == 1) & (df['has_internet'] == 1)).astype(int)

# Internet type flags
df['is_fiber'] = (df['InternetService'] == 'Fiber optic').astype(int)
df['is_dsl'] = (df['InternetService'] == 'DSL').astype(int)

print("📊 Core Service Flags:")
print(f"Has phone:         {df['has_phone'].sum():,} ({df['has_phone'].mean()*100:.1f}%)")
print(f"Has internet:      {df['has_internet'].sum():,} ({df['has_internet'].mean()*100:.1f}%)")
print(f"Has both core:     {df['has_both_core'].sum():,} ({df['has_both_core'].mean()*100:.1f}%)")
print(f"Is fiber:          {df['is_fiber'].sum():,} ({df['is_fiber'].mean()*100:.1f}%)")

📊 Core Service Flags:
Has phone:         6,361 (90.3%)
Has internet:      5,517 (78.3%)
Has both core:     4,835 (68.6%)
Is fiber:          3,096 (44.0%)


In [19]:
# Security services (protective add-ons)
df['has_security_services'] = ((df['OnlineSecurity'] == 'Yes') | 
                                (df['DeviceProtection'] == 'Yes') | 
                                (df['TechSupport'] == 'Yes')).astype(int)

# Streaming services (entertainment add-ons)
df['has_streaming'] = ((df['StreamingTV'] == 'Yes') | 
                        (df['StreamingMovies'] == 'Yes')).astype(int)

print("\n📊 Service Category Flags:")
print(f"Has security services: {df['has_security_services'].sum():,} ({df['has_security_services'].mean()*100:.1f}%)")
print(f"Has streaming:         {df['has_streaming'].sum():,} ({df['has_streaming'].mean()*100:.1f}%)")


📊 Service Category Flags:
Has security services: 3,779 (53.7%)
Has streaming:         3,499 (49.7%)


---

## 5. Feature Summary

In [20]:
# List all new features created
original_cols = ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 
                 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 
                 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 
                 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 
                 'numAdminTickets', 'numTechTickets', 'Churn']

new_features = [col for col in df.columns if col not in original_cols]

print("📊 NEW FEATURES CREATED")
print("=" * 50)
print(f"Total new features: {len(new_features)}")
print()

# Group by bucket
tenure_features = ['tenure_bucket', 'is_month_to_month', 'is_one_year', 'is_two_year', 
                   'has_contract_commitment', 'is_new_mtm_customer']
pricing_features = ['avg_monthly_charge', 'charge_tenure_ratio', 'is_high_charge', 
                    'is_low_charge', 'charge_quartile', 'is_electronic_check', 'has_auto_pay']
service_features = ['addon_count', 'has_any_addon', 'has_multiple_addons', 'no_addons',
                    'has_phone', 'has_internet', 'has_both_core', 'is_fiber', 'is_dsl',
                    'has_security_services', 'has_streaming']

print("🔹 Bucket 1: Tenure & Contract")
for f in tenure_features:
    print(f"   - {f}")

print("\n🔹 Bucket 2: Pricing Pressure")
for f in pricing_features:
    print(f"   - {f}")

print("\n🔹 Bucket 3: Service Dependency")
for f in service_features:
    print(f"   - {f}")

📊 NEW FEATURES CREATED
Total new features: 24

🔹 Bucket 1: Tenure & Contract
   - tenure_bucket
   - is_month_to_month
   - is_one_year
   - is_two_year
   - has_contract_commitment
   - is_new_mtm_customer

🔹 Bucket 2: Pricing Pressure
   - avg_monthly_charge
   - charge_tenure_ratio
   - is_high_charge
   - is_low_charge
   - charge_quartile
   - is_electronic_check
   - has_auto_pay

🔹 Bucket 3: Service Dependency
   - addon_count
   - has_any_addon
   - has_multiple_addons
   - no_addons
   - has_phone
   - has_internet
   - has_both_core
   - is_fiber
   - is_dsl
   - has_security_services
   - has_streaming


In [21]:
# Preview engineered dataset
print("\n📦 Final Dataset Shape:", df.shape)
df[['customerID', 'tenure', 'tenure_bucket', 'is_month_to_month', 'addon_count', 
    'MonthlyCharges', 'is_high_charge', 'Churn']].head(10)


📦 Final Dataset Shape: (7043, 47)


,customerID,tenure,tenure_bucket,is_month_to_month,addon_count,MonthlyCharges,is_high_charge,Churn
0,7590-VHVEG,1,1_New (0-12m),1,1,29.85,0,No
1,5575-GNVDE,34,3_Established (25-48m),0,2,56.95,0,No
2,3668-QPYBK,2,1_New (0-12m),1,2,53.85,0,Yes
3,7795-CFOCW,45,3_Established (25-48m),0,3,42.30,0,No
4,9237-HQITU,2,1_New (0-12m),1,0,70.70,0,Yes
5,9305-CDSKC,8,1_New (0-12m),1,3,99.65,1,Yes
6,1452-KIOVK,22,2_Developing (13-24m),1,2,89.10,0,No
7,6713-OKOMC,10,1_New (0-12m),1,1,29.75,0,No
8,7892-POOKP,28,3_Established (25-48m),1,4,104.80,1,Yes
9,6388-TABGU,62,4_Loyal (49+m),0,2,56.15,0,No


---

## 6. Save Engineered Data

In [22]:
# Save to processed folder
import os

output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'churn_features.csv')
df.to_csv(output_path, index=False)

print(f"✅ Engineered data saved to: {output_path}")
print(f"📦 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

✅ Engineered data saved to: ../data/processed\churn_features.csv
📦 Shape: 7,043 rows × 47 columns


---

## 📝 Summary

### Features Created by Bucket:

| Bucket | Feature Count | Key Features |
|--------|---------------|---------------|
| Tenure & Contract | 6 | `tenure_bucket`, `is_new_mtm_customer`, `has_contract_commitment` |
| Pricing Pressure | 7 | `charge_tenure_ratio`, `is_high_charge`, `has_auto_pay` |
| Service Dependency | 11 | `addon_count`, `has_security_services`, `has_streaming` |

### Design Principles:
1. ✅ **Hypothesis-driven**: Every feature tied to validated EDA findings
2. ✅ **Business interpretable**: Features can be explained to stakeholders
3. ✅ **No leakage risk**: All features use pre-churn information only
4. ✅ **Actionable**: Features suggest retention interventions

### Next Steps (03_modeling.ipynb):
- Train baseline models
- Feature importance analysis
- Model selection and tuning

---

*Notebook created: 2026-01-07*